# Assignment 02 - Machine Learning 2019/2020
Emanuel Fonseca - up200505480 PDEEC

***
***

### Exercise 1

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

***

### Exercise 1.a)

It will be calculated the w1 and w2 using the matrix formulation, as presented in the class:

$$\mathbf{w=(X^TX)^{-1}X^Ty}$$

In [2]:
data = np.array([[368, 15, 1.7], [340,16,1.5], [665,25,2.8],[954,40,5],[331,15,1.3]])
y = data[:,2].reshape(-1,1)
x = data[:,:2]

In [3]:
w = np.linalg.inv(x.T.dot(x)).dot(x.T).dot(y)

In [4]:
print('The w vector is: \n', w)

The w vector is: 
 [[0.00277318]
 [0.0487576 ]]


***

### Exercise 1.b)

If $X^TX$ is invertible than the solution of $f(y)$ is unique and to $X^TX$ be invertible, $det|X^TX|\neq0$.

Let's append the $x_3$ feature to the X matrix.

In [5]:
x3 = np.array([383, 356, 690, 994, 346]).reshape(-1,1)

In [6]:
x_b = np.append(x, x3, axis=1)

Now, lets calculate the determinant of $X^TX$:

In [7]:
print(np.linalg.det(x_b.T.dot(x_b)))

-0.004815404769033322


As it can be seen $det|X^TX|\neq0$, therefore $X^TX$ is invertible, so the solution of $f(y)$ is unique.

The solution for $f(y)$ is:

In [8]:
w_b = np.linalg.inv(x_b.T.dot(x_b)).dot(x_b.T).dot(y)

In [9]:
print('The w vector is: \n', w_b)

The w vector is: 
 [[-0.00145893]
 [ 0.0505187 ]
 [ 0.00269628]]


***
***

### Exercise 2

### Exercise 2.a)

See code comments for details.

In [2]:
data_arr = np.loadtxt('heightWeightData.txt', delimiter=',')

In [3]:
#Return the probability from the normal pdf
def normal_pdf(x, cov_det, mean_vector, cov_inv):
    return (1/math.sqrt((2*math.pi)**2*cov_det))*np.exp((-1/2)*np.diag((x-mean_vector).dot(cov_inv).dot((x-mean_vector).transpose())))

In [4]:
#return the classification (male/felame) using maximum a posterior probability
def classification_map(train_data, test_data):
    #calc of covariance matrix and mean for male data. (maximum likelihood estimation)
    male_data = train_data[train_data[:, 0]==1]
    male_mean = male_data[:,1:].mean(axis=0)
    male_cov = np.cov(male_data[:, 1:].transpose())
    male_cov_det = np.linalg.det(male_cov)
    male_cov_inv = np.linalg.inv(male_cov)
    #calc of covariance matrix and mean for female data. (maximum likelihood estimation)
    female_data = data_arr[data_arr[:, 0]==2]
    female_mean = female_data[:,1:].mean(axis=0)
    female_cov = np.cov(female_data[:, 1:].transpose())
    female_cov_det = np.linalg.det(female_cov)
    female_cov_inv = np.linalg.inv(female_cov)
    #prior probabilities
    p_male = len(male_data)/len(train_data)
    p_female = 1 - p_male
    #probability of data knowing is a male/female
    p_x_male = normal_pdf(test_data, male_cov_det, male_mean, male_cov_inv)
    p_x_female = normal_pdf(test_data, female_cov_det, female_mean, female_cov_inv)
    #probability of being a male/female knowing the data
    p_male_x = p_x_male * p_male
    p_female_x = p_x_female * p_female
    #classification process
    classification = np.ones(len(test_data))
    classification[p_male_x < p_female_x] = 2
    classification[p_male_x >= p_female_x] = 1
    
    return classification, p_male, p_female, p_x_male, p_x_female, p_male_x, p_female_x

***

In [6]:
test_points = np.array([[165, 80]])
classification_map(data_arr, np.array([[165, 80]]))

(array([1.]),
 0.3476190476190476,
 0.6523809523809524,
 array([7.91607213e-05]),
 array([3.99435247e-05]),
 array([2.75177745e-05]),
 array([2.60583947e-05]))

### Exercise 2.b)

Calculate $P([165, 80]^T|male)$ using the classification_map function.

In [13]:
p_x_male = classification_map(data_arr, np.array([[165, 80]]))[3]
print("P([165, 80]|male) = {}".format(p_x_male[0]))

P([165, 80]|male) = 7.916072126137533e-05


***

### Exercise 2.c)

note: the pandas lib was used to return a table-like format

In [14]:
test_points = np.array([[165, 80], [181, 65], [161, 57], [181, 77]])
classification = classification_map(data_arr, test_points)[0]
df_classification = pd.DataFrame({'Height': test_points[:,0], 'Weight': test_points[:,1], 'Class': classification})
df_classification.loc[df_classification.Class == 2, 'Class'] = 'female'
df_classification.loc[df_classification.Class == 1, 'Class'] = 'male'
print(df_classification)

   Height  Weight   Class
0     165      80    male
1     181      65    male
2     161      57  female
3     181      77    male


***

### Exercise 2.d)

Compute the D and S directly from the data array.

In [15]:
d_s_data = data_arr.copy()
d_s_data[:, 1] = data_arr[:, 1] - data_arr[:, 2]
d_s_data[:, 2] = (data_arr[:, 1] + data_arr[:, 2]) / 2

In [16]:
p_x_male = classification_map(d_s_data, np.array([[165-80, (165+80)/2]]))[3]
print("P([165, 80]|male) = {}".format(p_x_male[0]))

P([165, 80]|male) = 7.916072126137416e-05


In [17]:
test_points_d_s = test_points.copy()
test_points_d_s[:, 0] = test_points[:, 0] - test_points[:, 1]
test_points_d_s[:, 1] = (test_points[:, 0] + test_points[:, 1]) / 2
classification = classification_map(d_s_data, test_points_d_s)[0]
df_classification = pd.DataFrame({'D': test_points_d_s[:,0], 'S': test_points_d_s[:,1], 'Class': classification})
df_classification.loc[df_classification.Class == 2, 'Class'] = 'female'
df_classification.loc[df_classification.Class == 1, 'Class'] = 'male'
print(df_classification)

     D    S Class
0   85  122  male
1  116  123  male
2  104  109  male
3  104  129  male


***

### Exercise 2.e)

In [18]:
import scipy.linalg as linalg

We need to find $[z_1 z_2]^T = C[h w]^T$, where $\sum(C[h w]^T) = I$

Knowing that $\sum(CX) = E(CXX^TC^T)$, where $X = [h w]^T$. Then $\sum(CX) = C\sum(X)C^T$.

Assuming that $C = C^T$, doing the "algebra" we get:

$$C = (\sum X)^{-1/2}$$


In [19]:
cov = np.cov(data_arr[:,1:].transpose())
c = linalg.sqrtm(np.linalg.inv(cov))
y = c.dot(data_arr[:,1:].transpose())

In [20]:
print("As shown the new covariance is:\n", np.cov(y))

As shown the new covariance is:
 [[1.00000000e+00 2.34774163e-16]
 [2.34774163e-16 1.00000000e+00]]
